# 10.MNIST 손글씨 데이터 분류 (MLP 버전)
### 미 국립표준기술원(NIST)이 고등학생과 인구조사국 직원 등이 직접 쓴 손글씨 데이터
* 70,000 건의 0~9까지 흑백 손글씨 숫자 이미지 데이터
    - 60,000 건은 훈련 데이터로
    - 10,000 건은 테스트 데이터로 지정되어 있음

In [ ]:
from keras.datasets import mnist
from keras.utils import np_utils

import numpy as np

### MNIST 데이터셋 불러오기
MNIST 데이터셋은 NumPy 배열 형태로 케라스에 포함되어 있음

In [ ]:
(X_train, Y_class_train), (X_test, Y_class_test) = mnist.load_data()

print(f"학습셋 이미지 수 : {X_train.shape[0]} 개")
print(f"테스트셋 이미지 수 : {X_test.shape[0]} 개")

### 첫 번째 이미지 확인

In [ ]:
type(X_train)

In [ ]:
X_train.shape

In [ ]:
X_train[0].shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[0], cmap='gray')

In [ ]:
plt.imshow(X_train[0], cmap='Greys')

In [ ]:
X_train[0]

In [ ]:
Y_class_train[0]

### 이미지를 값으로 확인

In [ ]:
import sys
for x in X_train[0]:
    for i in x:
        sys.stdout.write(f'{i:4}')
    sys.stdout.write('\n')

### 신경망의 입력 데이터는 모든 값을 0과 1 사이로 스케일을 조정

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 784)
X_train = X_train.astype('float64')
X_train = X_train / 255

X_test = X_test.reshape(X_test.shape[0], 784).astype('float64') / 255

print(X_train[0])

### 첫 번째 이미지의 클래스 레이블 확인

In [ ]:
print(f"class : {Y_class_train[0]}")

### 원 핫 인코딩으로 변환

In [ ]:
Y_train = np_utils.to_categorical(Y_class_train, 10)
Y_test = np_utils.to_categorical(Y_class_test, 10)

print(Y_train[0])

### MLP 모델 생성

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=784, activation='relu'))
model.add(Dense(10, activation='softmax'))

# 모델 실행 환경 설정
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### 모델 학습 자동 중단 설정

In [ ]:
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

### 모델 학습 수행

In [ ]:
history = model.fit(X_train, Y_train, validation_split=0.2, epochs=30, batch_size=200, verbose=0, callbacks=[early_stopping_callback, checkpointer])

### 테스트 데이터 셋으로 모델의 성능 평가

In [ ]:
X_test.shape

In [ ]:
Y_test.shape

In [ ]:
X_test[0]

In [ ]:
history.history['acc'][-1]

In [ ]:
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

### 결과 검토

In [ ]:
# 검증 셋의 오차와 정확도
y_vloss = history.history['val_loss']
y_vacc = history.history['val_acc']

# 학습셋의 오차와 정확도
y_loss = history.history['loss']
y_acc = history.history['acc']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Validation_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Training_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='best')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
# 그래프로 표현
x_len = np.arange(len(y_acc))
plt.plot(x_len, y_vacc, marker='.', c="red", label='Validation_acc')
plt.plot(x_len, y_acc, marker='.', c="blue", label='Training_acc')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='best')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')

### 훈련 세트의 정확도와 검증 세트의 정확도의 갭이 크지 않음 ==> 과적합 아님
### 테스트 세트의 정확도가 높다 ==> 일반화가 잘 됨

### 잘 학습된 것으로 판단된 모델을 저장

In [ ]:
model.save('./mnist_model.h5')